In [28]:
from datetime import datetime

class Forum:
    messages = []

    def init(self):
        self.messages = []

    def recupDate(self):
        dateBrute = str(datetime.now())
        date = dateBrute[11:13] + "h" + dateBrute[14:16] + "m" + dateBrute[17:19] + "s"
        return date

    def calculDate(self, date2):
            msg_date = date2.split()
            date = ""
            i=0
            while msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            heure = int(date[0:2])
            while msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            minute = int(date[2:4])
            while len(msg_date) and msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            seconde = int(date[4:6])
            total = 3600*heure+60*minute+seconde
            return (total)
    
    def recupererMessagesAnonymes(self, date_debut, date_fin) :
        for msg in self.messages :
            date = self.calculDate(msg[1])
            date_start = self.calculDate(date_debut)
            date_end = self.calculDate(date_fin)
            if date > date_start and date < date_end :
                print(msg)

    def posterMessageAnonyme(self, message) :
        date = self.recupDate()
        self.messages.append([message, date])



In [29]:
if __name__ == "__main__" :
    print("start")
    forum = Forum()
    while(True) :
        string = input()
        if "show_message" == string :
            print("entrez la date au format 11h02m03s" )
            date_debut = input()
            date_fin = input()
            forum.recupererMessagesAnonymes(date_debut, date_fin)
        else :
            forum.posterMessageAnonyme(string)

start
entrez la date au format 11h02m03s



ValueError: invalid literal for int() with base 10: ''

# <b> Canal de diffusion anonyme et génération de secret </b>

<i> Compte-rendu du TP2 par Valentin SÉRÉE et Jérémy SERENA [INFO3]</i>